# 车牌字符分割

In [2]:
import numpy as np
import sys
import os
import cv2

import matplotlib.pyplot as plt

from skimage import io,filters
from skimage.transform import resize

In [3]:
GRAYED_DIR = "./outputs/Grayed_Image"
CROPPED_DIR = "./outputs/Cropped_Image"
PLATE_SEG_CHARS = "./outputs/Plate_Seg_Chars"

In [4]:
CHAR_HEIGHT = 92
CHAR_WIDTH = 47

## 去除车牌边框

对车牌二值化图像行和列扫描。当行扫描时跳变小于某个阈值时，则认为这行数据中不包含字符信息，这行的像素直接置0。这样去除掉上、下边框和铆钉的影响。当扫描完行之后，直接在前端和末端抹掉两列（经验值）信息。这是为了去掉左右边框的影响。

In [53]:
for i in range(1,5):
    path = os.path.join(GRAYED_DIR, "gray_"+str(i)+".png")
    img = io.imread(path, as_gray=True)
    rows, cols = img.shape

    first = True
    count_b = 0
    count_p = 0
    upside = 0
    downside = rows
    
    for m in range(1, int(rows/3)):
        sum_row = np.sum(img[m]) / 255 / cols
        if sum_row>0.2 and sum_row<0.6 and first:
            count_b = m
            first = False
            continue
            
        if sum_row>0.2 and sum_row<0.6 and count_b==m-1:
            count_b = m
            count_p += 1
        else:
            first = True
            count_b = 0
            count_p = 0
    
        if count_p >= 7:
            first = True
            upside = m-15
            break
    
    for m in range(rows-1, rows-int(rows/3), -1):
        sum_row = np.sum(img[m]) / 255 / cols
        if sum_row>0.2 and sum_row<0.7 and first:
            count_b = m
            first = False
            continue
            
        if sum_row>0.2 and sum_row<0.7 and count_b==m+1:
            count_b = m
            count_p += 1
        else:
            first = True
            count_b = 0
            count_p = 0
    
        if count_p >= 7:
            first = True
            downside = m+15
            break

    print("up is %g, down is %g" %(upside, downside))
    
    _img = img[upside:downside, 5:-5]
    thresh = filters.threshold_otsu(_img)
    dst = (_img > thresh) * 1
    if not os.path.exists(CROPPED_DIR):
        os.mkdir(CROPPED_DIR)     
    path_out = os.path.join(CROPPED_DIR, "a_"+str(i)+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)   

up is 17, down is 93
up is 16, down is 136
up is 7, down is 104
up is 2, down is 118


for i in range(1,151):
    path = os.path.join(GRAYED_DIR, "gray_"+str(i)+".png")
    img = io.imread(path, as_gray=True)
    
    rows, cols = img.shape
    first = True
    count_b = 1
    count_p = 0
    upside = 0
    downside = rows
    
    for m in range(1, int(rows/5)):
        sum_row = np.sum(img[m]) / 255 / cols
        if sum_row>0.3 and sum_row<0.6 and first:
            count_b = m
            first = false
        
        if sum_row>0.3 and sum_row<0.6 and count_b==m-1:
            count_b = m
            count_p += 1
        else:
            first = True
            count_b = 0
            count_p = 0
    
        if count_p >= 15:
            first = True
            upside = m-15
            
    
    
    idxs = np.concatenate((range(0,int(rows/10)), range(rows-int(rows/9),rows)))

    for k in idxs:
        sum_row = np.sum(img[k]) / 255 / cols
        if sum_row<0.35 or sum_row > 0.5:
            img[k] = 0

    _img = img[:, 5:-5]
    thresh = filters.threshold_otsu(_img)
    dst = (_img > thresh) * 1
    if not os.path.exists(CROPPED_DIR):
        os.mkdir(CROPPED_DIR)     
    path_out = os.path.join(CROPPED_DIR, "crop_"+str(i)+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)

## 字符截取

In [8]:
for i in range(1,151):
    path = os.path.join(CROPPED_DIR, "crop_"+str(i)+".png")
    
    dir_out = os.path.join(PLATE_SEG_CHARS,"Plate_" +str(i))
    if not os.path.exists(dir_out):
        os.mkdir(dir_out)
 
    img = io.imread(path, as_gray=True)
    rows, cols = img.shape
    per = np.floor(cols/7.5).astype(int)
    for k in range(7):
        if k > 1:
            idxs = range(int((k+0.5)*per), int((k+1.5)*per))
        else:
            idxs = range(k*per, (k+1)*per)
        char = resize(img[:,idxs], (CHAR_HEIGHT, CHAR_WIDTH), mode='reflect')
        path_out = os.path.join(dir_out,"char_"+str(k+1)+".png")
        io.imsave(path_out, char)
#         plt.figure()
#         io.imshow(char)
#         plt.show()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


### 手动矫正

对异常的车牌进行手动处理

In [24]:
input1_ = []
input2_ = []
with open("./datasets/Char_Seg_Failed.txt") as file_idx_err:
    while True:
        # 整行读取数据
        lines = file_idx_err.readline()
        if not lines:
            break
        # 将整行数据分割处理，这里分隔符为空格
        tmp1, tmp2 = [i for i in lines.split()]
        input1_.append(tmp1)
        input2_.append(tmp2)

    input1_ = np.array(input1_[1:])
    input2_ = np.array(input2_[1:])
    IndexFileName_Err = input2_

for n in range(0, len(IndexFileName_Err)):
    path_e = os.path.join(GRAYED_DIR, "bin_"+str(IndexFileName_Err[n])+".png")
    img = io.imread(path_e, as_gray=True)
    rows, cols = img.shape
    idxs = np.concatenate((range(0,int(rows/10)), range(rows-int(rows/9),rows)))

    for k in idxs:
        sum_row = np.sum(img[k]) / 255 / cols
        if sum_row<0.35 or sum_row > 0.5:
            img[k] = 0

    _img = img[:, 2:-5]
    thresh = filters.threshold_otsu(_img)
    dst = (_img > thresh) * 1
    if not os.path.exists(CROPPED_DIR):
        os.mkdir(CROPPED_DIR)   
    path_out = os.path.join(CROPPED_DIR, "crop_"+str(IndexFileName_Err[n])+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)
#     plt.figure()
#     io.imshow(dst)
#     plt.show()

./outputs/Grayed_Image/bin_25.png
